In [1]:
! dir G:\AzureBackup\Datasets\Twitter\microposts-NEEL\2016

 Volume in drive G is Dev
 Volume Serial Number is B036-DF93

 Directory of G:\AzureBackup\Datasets\Twitter\microposts-NEEL\2016

12/05/2018  07:50 PM    <DIR>          .
12/05/2018  07:50 PM    <DIR>          ..
12/05/2018  07:50 PM           173,346 all_ids.txt
08/26/2016  11:42 AM               206 AUTHOR.txt
08/26/2016  12:28 PM             1,900 microposts2016-neel-dev-tweets-ids.tsv
01/31/2016  12:59 AM            23,440 microposts2016-neel-dev_neel.gs
08/26/2016  12:32 PM               344 microposts2016-neel-README
08/26/2016  12:29 PM            60,116 microposts2016-neel-test-tweets-ids.tsv
04/06/2016  07:46 AM            72,781 microposts2016-neel-test_neel.gs
08/26/2016  12:29 PM           111,330 microposts2016-neel-training-tweets-ids.tsv
01/31/2016  01:21 AM           593,313 microposts2016-neel-training_neel.gs
09/29/2016  05:07 PM           225,865 microposts2016-neel_challenge_gs.zip
07/09/2015  07:17 AM            13,400 microposts2016-neel_challenge_gs_cc-by4.0_lice

In [2]:
! dir G:\AzureBackup\Datasets\Twitter\microposts-NEEL\processed\2016\

 Volume in drive G is Dev
 Volume Serial Number is B036-DF93

 Directory of G:\AzureBackup\Datasets\Twitter\microposts-NEEL\processed\2016

04/16/2019  03:41 PM    <DIR>          .
04/16/2019  03:41 PM    <DIR>          ..
04/16/2019  04:21 PM            18,954 microposts2016-neel-dev_neel.gs.conll
04/16/2019  04:21 PM             1,760 microposts2016-neel-dev_neel.gs.conll.tweet_ids
04/16/2019  04:21 PM            25,436 microposts2016-neel-dev_neel.gs.jsonl
04/16/2019  04:21 PM           481,076 microposts2016-neel-test_neel.gs.conll
04/16/2019  04:21 PM            53,260 microposts2016-neel-test_neel.gs.conll.tweet_ids
04/16/2019  04:21 PM           219,049 microposts2016-neel-test_neel.gs.jsonl
04/16/2019  04:21 PM           474,228 microposts2016-neel-training_neel.gs.conll
04/16/2019  04:21 PM            49,480 microposts2016-neel-training_neel.gs.conll.tweet_ids
04/16/2019  04:21 PM           811,573 microposts2016-neel-training_neel.gs.jsonl
               9 File(s)      2,134,

In [3]:
def read_sequences(fp):
    for seq_str in fp.read().split("\n\n"):
        if seq_str:
            yield [line.split("\t") for line in seq_str.splitlines()]

In [4]:
import pandas as pd

import os
import json

In [5]:
RAW_BASE_DIR="G:\\AzureBackup\\Datasets\\Twitter\\microposts-NEEL\\"
with open(os.path.join(RAW_BASE_DIR, "all_ids.txt.json")) as fp:
    tweet_json = {}
    for line in fp:
        line = json.loads(line)
        tweet_json[line["id_str"]] = line

In [6]:
BASE_PATH = "G://AzureBackup//Datasets//Twitter//microposts-NEEL\\processed\\2016\\"
for prefix in ["training", "dev", "test"]:
    print(prefix)
    with open(os.path.join(RAW_BASE_DIR, "processed\\2016", f"microposts2016-neel-{prefix}_neel.gs.conll"), encoding="utf-8") as fp:
        sequences = list(read_sequences(fp))
        df = pd.DataFrame([
            [seq_id, line_id]+line
            for seq_id, seq in enumerate(sequences)
            for line_id, line in enumerate(seq)
        ], columns=["seq_id", "line_id", "token", "label"])
        df = pd.concat([
            df, 
            df.label.str.rsplit("-", expand=True).rename(columns={0: "boundary", 1: "label_type"})
        ], axis=1)
        print(df[(df.boundary == "B") & (~df.token.str.startswith(("@", "#")))].label.value_counts())
    with open(os.path.join(BASE_PATH, f"microposts2016-neel-{prefix}_neel.gs.conll.tweet_ids"), encoding="utf-8") as fp:
        tweet_ids = [line.strip() for line in fp.readlines()]

training
B-Organization    664
B-Person          656
B-Location        581
B-Product         266
B-Thing           136
B-Event            96
B-Character        34
Name: label, dtype: int64
dev
B-Product         19
B-Person          14
B-Thing           10
B-Organization    10
B-Character        9
B-Location         6
B-Event            1
Name: label, dtype: int64
test
B-Product         71
B-Person          48
B-Character       44
B-Organization    28
B-Thing           28
B-Location        20
B-Event           11
Name: label, dtype: int64


In [7]:
len(sequences)

2663

In [8]:
len(tweet_ids)

2663

In [9]:
df = pd.DataFrame([
    [seq_id, tweet_id, line_id]+line
    for seq_id, (tweet_id, seq) in enumerate(zip(tweet_ids, sequences))
    for line_id, line in enumerate(seq)
], columns=["seq_id", "tweet_id", "line_id", "token", "label"])
df = pd.concat([
    df, 
    df.label.str.rsplit("-", expand=True).rename(columns={0: "boundary", 1: "label_type"})
], axis=1)

In [10]:
df.head()

,seq_id,tweet_id,line_id,token,label,boundary,label_type
0,0,674963312203816962,0,Day,O,O,None
1,0,674963312203816962,1,9,O,O,None
2,0,674963312203816962,2,:,O,O,None
3,0,674963312203816962,3,Wearing,O,O,None
4,0,674963312203816962,4,a,O,O,None


In [11]:
df[df.boundary == "B"].label.value_counts()

B-Product         308
B-Person          278
B-Organization    138
B-Character        53
B-Thing            44
B-Location         34
B-Event            21
Name: label, dtype: int64

In [12]:
df_annotation = pd.read_csv(
    os.path.join(RAW_BASE_DIR, "2016", "microposts2016-neel-test_neel.gs"), sep="\t", header=None
)
df_annotation.columns = [f"col_{c}" for c in df_annotation.columns]
df_annotation.head()

,col_0,col_1,col_2,col_3,col_4,col_5
0,674869443671941120,93,101,http://dbpedia.org/resource/Egyptians,1,Thing
1,674869443671941120,63,68,http://dbpedia.org/resource/Barack_Obama,1,Person
2,674869443671941120,112,123,NIL101,1,Person
3,674869443671941120,4,17,NIL113,1,Person
4,674869443671941120,125,135,NIL165,1,Person


In [13]:
df[(df.boundary == "B") & (~df.token.str.startswith(("@", "#")))].head(20)

,seq_id,tweet_id,line_id,token,label,boundary,label_type
642,33,675578093348642817,1,Star,B-Product,B,Product
659,33,675578093348642817,18,pisode,B-Product,B,Product
660,34,675295448568057856,0,eyerizzz,B-Person,B,Person
724,38,675228549419257856,7,Pixel,B-Person,B,Person
1536,81,675577399472291840,17,WW2,B-Event,B,Event
1543,82,676784260880166912,0,Hollywood,B-Location,B,Location
1552,82,676784260880166912,9,the,B-Organization,B,Organization
1855,101,675577451838337024,3,trump,B-Person,B,Person
1971,108,675563968816144384,4,Paris,B-Person,B,Person
2195,122,674966858219413504,4,REY,B-Character,B,Character


In [49]:
inspect_idx = 166
sequences[inspect_idx]

[['RT', 'O'],
 ['@HereIsGina', 'B-Person'],
 [':', 'O'],
 ['“', 'O'],
 ['@bexpretty', 'B-Person'],
 [':', 'O'],
 ['@HereIsGina', 'B-Person'],
 ['hey', 'O'],
 ['😜', 'O'],
 ['👏', 'O'],
 ['🏻', 'O'],
 ['awesome', 'O'],
 ['!', 'O'],
 ['!', 'O'],
 ['!', 'O'],
 ["Who's", 'O'],
 ['your', 'O'],
 ['fav', 'O'],
 ['character', 'O'],
 ['?', 'O'],
 ['#Sta', 'O'],
 ['rWars', 'B-Product'],
 ['xo', 'I-Product'],
 ['xo', 'O'],
 ['💋', 'O'],
 ['”', 'O'],
 ['difficult', 'O'],
 ['@John', 'O'],
 ['Boyega', 'B-Person'],
 ['#Da', 'I-Person'],
 ['is', 'O'],
 ['yRidley', 'B-Person'],
 ['and', 'I-Person'],
 ['…', 'O']]

In [50]:
text = tweet_json[tweet_ids[inspect_idx]]["full_text"]
print(text)

RT @HereIsGina: “@bexpretty: @HereIsGina hey😜👏🏻 awesome!!! Who's your fav character? #StarWars xoxo💋”difficult @JohnBoyega #DaisyRidley and…


In [51]:
df_annotation[df_annotation["col_0"] == int(tweet_ids[inspect_idx])].sort_values("col_1")

,col_0,col_1,col_2,col_3,col_4,col_5
877,676802417690849280,4,14,NIL118,1,Person
878,676802417690849280,18,27,NIL90,1,Person
876,676802417690849280,30,40,NIL118,1,Person
873,676802417690849280,89,97,http://dbpedia.org/resource/Star_Wars,1,Product
874,676802417690849280,116,126,http://dbpedia.org/resource/John_Boyega,1,Person
875,676802417690849280,128,139,http://dbpedia.org/resource/Daisy_Ridley,1,Person


In [52]:
for i, row in df_annotation[df_annotation["col_0"] == int(tweet_ids[inspect_idx])].sort_values("col_1").iterrows():
    print(text[row["col_1"]:row["col_2"]])

HereIsGina
bexpretty
HereIsGina
rWars xo
Boyega #Da
yRidley and
